<a href="https://colab.research.google.com/github/mc-friday/hanghaeAI/blob/main/%5B4%EC%A3%BC%EC%B0%A8%5D%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [4주차] 심화과제: 수능 국어 문제 GPT-4로 풀어보기

In [1]:
!pip install openai

In [2]:
import openai
import json
import time

openai.api_key = "key"
client = openai.OpenAI(api_key=openai.api_key)

## 1. 수능 문제 JSON 데이터 로드 함수

In [3]:
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

## 2. 하나의 문제에 대해서 GPT-4의 예측 결과를 내놓는 함수

In [4]:
def prediction(problem):
    """
    문제 데이터를 받아 GPT-4가 답안을 예측하도록 하는 함수
    """
    zero_shot_cot_en_prompt_plus = f"""
    국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.
    지문을 읽고, 질문에 대한 답을 1부터 5까지의 선택지 중에 한 개만 골라서 대답해야 합니다.
    <보기>가 있으면 꼭 문제 풀이에 참고해야 합니다.

    지문 :
    {problem['paragraph']}

    질문 :
    {problem['question']}

    <보기> :
    {problem.get('question_plus', '')}

    선택지 :
    1 - {problem['choices'][0]}
    2 - {problem['choices'][1]}
    3 - {problem['choices'][2]}
    4 - {problem['choices'][3]}
    5 - {problem['choices'][4]}

    1번, 2번, 3번, 4번, 5번 중에 하나를 정답으로 고르세요.
    단계별로 생각하며 정답을 고르세요.
    정답은 숫자만 답하세요.
    정답:
    """

     # 문제 풀이 시작 시간 기록
    start_time = time.time()


    # GPT-4 API 호출
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": zero_shot_cot_en_prompt_plus}
            ]
        )
        # GPT-4의 답변에서 선택한 답안을 반환
        predicted_answer = response.choices[0].message.content

         # 문제 풀이 종료 시간 기록
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f"[LOG] 문제 풀이 시간: {elapsed_time:.2f}초, 예측된 정답: {predicted_answer}")

        return predicted_answer
    except Exception as e:
        print(f"Error during GPT-4 prediction: {e}")
        return None


## 3. 모든 문제를 평가하고 점수를 계산하는 함수

In [5]:
def evaluate_all_problems(data):
    """
    JSON 데이터의 모든 문제를 평가하고 점수를 계산하는 함수
    """
    total_score = 0  # 총 점수
    possible_score = 0  # 가능한 총 점수
    correct_count = 0  # 정답 개수
    total_count = 0  # 전체 문제 수

    for item in data:
        paragraph = item["paragraph"]
        for problem in item["problems"]:
            total_count += 1
            possible_score += problem["score"]  # 가능한 점수 누적

            # 문제 데이터 준비
            problem_data = {
                "paragraph": paragraph,
                "question": problem["question"],
                "choices": problem["choices"],
                "question_plus": problem.get("question_plus", ""),
            }

            # GPT-4 답안 예측
            predicted_answer = prediction(problem_data)

            # 실제 정답과 비교
            if str(predicted_answer) == str(problem["answer"]):  # 정답 비교
                total_score += problem["score"]
                correct_count += 1

    # 결과 출력
    print(f"총 문제 수: {total_count}")
    print(f"정답 개수: {correct_count}/{total_count}")
    print(f"정확도: {correct_count / total_count:.2%}")
    print(f"총 점수: {total_score}/{possible_score}")
    return total_score, possible_score, correct_count, total_count

## 4. 평가 실행

In [6]:
file_path = "./data/2023_11_KICE.json"  # Colab에 업로드한 JSON 파일 경로

# 데이터 로드
data = load_data(file_path)

# 모든 문제 평가 실행
total_score, possible_score, correct_count, total_count = evaluate_all_problems(data)

[LOG] 문제 풀이 시간: 0.51초, 예측된 정답: 4
[LOG] 문제 풀이 시간: 0.43초, 예측된 정답: 5
[LOG] 문제 풀이 시간: 0.56초, 예측된 정답: 1
[LOG] 문제 풀이 시간: 0.57초, 예측된 정답: 4
[LOG] 문제 풀이 시간: 0.76초, 예측된 정답: 5
[LOG] 문제 풀이 시간: 0.51초, 예측된 정답: 3
[LOG] 문제 풀이 시간: 0.64초, 예측된 정답: 2
[LOG] 문제 풀이 시간: 1.02초, 예측된 정답: 3
[LOG] 문제 풀이 시간: 0.48초, 예측된 정답: 2
[LOG] 문제 풀이 시간: 2.19초, 예측된 정답: 1

선택지 1번은 틀렸습니다. 법령의 요건과 효과 모두에 불확정 개념이 사용될 수 있다는 주장은 지문에서 지지되지 않으며, 이러한 공통적 특성이 조문에 항상 나타나는 것으로 설명하지 않습니다.
[LOG] 문제 풀이 시간: 2.95초, 예측된 정답: 5
[LOG] 문제 풀이 시간: 0.52초, 예측된 정답: 2
[LOG] 문제 풀이 시간: 2.68초, 예측된 정답: 1
[LOG] 문제 풀이 시간: 2.80초, 예측된 정답: 3
[LOG] 문제 풀이 시간: 3.83초, 예측된 정답: 5

(클라이버의 법칙에 따라 기초 대사량은 (체중)^0.75에 비례하며, 이는 초기 연구에서 바랐던 (체중)^0.67에 비례하는 것과 다르지만, 체중이 증가하면 대사량 증가율은 초기 연구자들이 예상한 것보다 더 낮습니다. 따라서 예상보다 많은 양의 에너지가 필요하지 않다는 것이 적절하지 않은 추론입니다.)
[LOG] 문제 풀이 시간: 0.62초, 예측된 정답: 4
[LOG] 문제 풀이 시간: 0.60초, 예측된 정답: 1
[LOG] 문제 풀이 시간: 0.61초, 예측된 정답: 4
[LOG] 문제 풀이 시간: 0.86초, 예측된 정답: 3
[LOG] 문제 풀이 시간: 0.45초, 예측된 정답: 3
[LOG] 문제 풀이 시간: 0.50초, 예측된 정답: 5
[LOG] 문제 풀이 시간: 1.20초, 예측된 정답